<a href="https://colab.research.google.com/github/sdgroeve/EuBIC2022_workshop_ML/blob/main/mnist_digit_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Loading data with Tensorflow Keras

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Defining the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

print(model.summary())

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Fitting the data

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

### Making predictions

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predicted_classes = predictions.argmax(axis=-1)
print(predicted_classes)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred):
    mtx = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(8,8))
    sns.heatmap(mtx, annot=True, fmt='d', linewidths=.5,  cbar=False, ax=ax)
    plt.ylabel('true label')
    plt.xlabel('predicted label')

y_test_classes = y_test.argmax(axis=-1)
plot_confusion_matrix(y_test_classes, predicted_classes)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_classes, predicted_classes))

In [ ]:
from keras.preprocessing import image

!wget https://raw.githubusercontent.com/sdgroeve/EuBIC2022_workshop_ML/main/chicken.jpg

img = image.load_img('chicken.jpg')
plt.imshow(img)

In [ ]:
import pandas as pd

img = image.load_img('chicken.jpg', target_size=(28, 28), color_mode="grayscale")
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
predictions = model.predict(images)

pd.DataFrame(predictions).plot.bar()
plt.show()